In [1]:
import numpy as np 
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt 
from xml.dom import minidom
import csv
import time as tm
import datetime

class TouchPair:
	def __init__(self, classification, orientation, distance, time, paramIndexTouchPoint, paramIndexLastPoint):
		self.classification = classification
		self.orientation = orientation
		self.distance = distance
		self.time = time
		self.paramIndexTouchPoint = paramIndexTouchPoint
		self.paramIndexLastPoint = paramIndexLastPoint

# form a touch pair with 3 calculated features
def pair(touchPoint, lastPoint, indexInLast, indexTouch):
	orientation = round((abs(float(touchPoint[1]) - float(lastPoint[1]))) % 1, 5)
	yDistance = abs(float(touchPoint[2]) - float(lastPoint[2]))
	xDistance = abs(float(touchPoint[3]) - float(lastPoint[3]))
	distance = round(np.sqrt(yDistance*yDistance + xDistance*xDistance) / 1557, 3)
	time = (touchPoint[4] - lastPoint[4])*1000
	global deltaTimeList
	deltaTimeList.append(time)

	touchPoint[5] = indexTouch
	global lastPoints
	if touchPoint[0] == lastPoint[0]:
		classification = "same"
		lastPoints[indexInLast] = touchPoint
		# print("same")
	else:
		classification = "different"
		flagNewUser = True
		if (len(lastPoints)) > 1:
			for i in range(len(lastPoints)):
				if touchPoint[0] == lastPoints[i][0]:
					flagNewUser = False
					# print("flag")
		if flagNewUser == True:
			lastPoints.append(touchPoint)
	# 	print("different")
	# 	print(touchPoint[0])
	# 	print(lastPoint[0])
	# 	print(lastPoints)
	# print("------------------")
	touchPair = TouchPair(classification, orientation, distance, time, indexTouch, lastPoint[5])
	global touchPairs
	touchPairs.append(touchPair)	

if __name__ == "__main__":
	# read data from xml files
	indexNow = 0
	nameList = ["1 - Brainstorm", 
				"2 - Heuristics", "3 - Heuristics", "4 - Heuristics", "5 - Heuristics", 
				"6 - Map", "7 - Map", "8 - Map", "9 - Map",
				"10 - Resources", "11 - Resources", "12 - Resources", "13 - Resources",
				"14 - Wireframes", "15 - Wireframes", "16 - Wireframes", "17 - Wireframes"]
	xmldoc = minidom.parse(nameList[indexNow]+".xml")
	logs = xmldoc.getElementsByTagName('Point')
	touchPoints = []
	deltaTimeList = []
	# remove outliers with elapsed time between touches above 80% of full dataset
	# for i in range(300):
	for i in range(len(logs)):
		if '?' not in logs[i].attributes['student'].value and logs[i].attributes['student'].value != "":
			if i == 0:
				tempTimeStr = logs[0].attributes['timestamp'].value
				tempTimeFormat = datetime.datetime.strptime(tempTimeStr, "%Y/%m/%d %H:%M:%S:%f")
				tempTime = tm.mktime(tempTimeFormat.timetuple())+(tempTimeFormat.microsecond/1000000.0)
				thisTime = tempTime
			else:
				thisTimeStr = logs[i].attributes['timestamp'].value
				thisTimeFormat = datetime.datetime.strptime(thisTimeStr, "%Y/%m/%d %H:%M:%S:%f")
				thisTime = tm.mktime(thisTimeFormat.timetuple())+(thisTimeFormat.microsecond/1000000.0)
			deltaTime = (thisTime - tempTime)*1000
			deltaTimeList.append(deltaTime)
			tempTime = thisTime
			touchPoints.append([logs[i].attributes['student'].value,
					logs[i].attributes['o'].value,
					logs[i].attributes['y'].value,
					logs[i].attributes['x'].value,
					thisTime,
					deltaTime])
	deltaTimeList = sorted(deltaTimeList, reverse=True)
	deltaTimeThreshold = deltaTimeList[int(0.2*len(deltaTimeList))]
	
	countDel = 0
	for i in range(len(touchPoints)-1):
		if touchPoints[i-countDel][5] > deltaTimeThreshold:
			del touchPoints[i-countDel]
			countDel += 1

	# global touchPairs
	touchPairs = []

	# initialize lastPoints with no.1 touchpoint
	# list columns are in order of student, o, y, x, timestamp, indexInTouchPoints
	lastPoints = [[touchPoints[0][0],
					touchPoints[0][1],
					touchPoints[0][2],
					touchPoints[0][3],
					touchPoints[0][4],
					0]]

	countDiff = 0

	for i in range(1, len(touchPoints)):
		print("---------")
		print(touchPoints[i])
		if len(lastPoints) == 1:
			pair(touchPoints[i], lastPoints[0], 0, i)
		else:
			for j in range(len(lastPoints)):
				pair(touchPoints[i], lastPoints[j], j, i)

	# normalize time feature and delete outliers
	deltaTimeList = sorted(deltaTimeList, reverse=True)
	deltaTimeThreshold = deltaTimeList[int(0.2*len(deltaTimeList))-1]

	countDel = 0
	for i in range(len(touchPairs)):
		if touchPairs[i-countDel].time > deltaTimeThreshold:
			del touchPairs[i-countDel]
			countDel += 1
		else:
			touchPairs[i-countDel].time = round(touchPairs[i-countDel].time / deltaTimeThreshold, 5)

	with open(nameList[indexNow]+" - touchPairs.csv","a",newline="") as fp:
		writer = csv.writer(fp, dialect='excel', delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_ALL)
		writer.writerow(["classification", "orientation", "distance", "time", "indexTouchPoint", "indexLastPoint"])
		for i in range(len(touchPairs)):
			writer.writerow([touchPairs[i].classification,
							touchPairs[i].orientation,
							touchPairs[i].distance,
							touchPairs[i].time,
							touchPairs[i].paramIndexTouchPoint,
							touchPairs[i].paramIndexLastPoint])
	fp.close()

	# train MLP using leave-one-out nested cross validation in wek 3.8

---------
['teacher', '5.85952', '634', '1100', 1404904442.465, 11.999845504760742]
---------
['teacher', '5.63806', '631.667', '1097.67', 1404904442.482, 17.000198364257812]
---------
['teacher', '5.63806', '630.5', '1095', 1404904442.498, 15.999794006347656]
---------
['teacher', '5.63806', '629.4', '1091.8', 1404904442.515, 17.000198364257812]
---------
['teacher', '5.63806', '628.333', '1088.17', 1404904442.531, 15.999794006347656]
---------
['teacher', '5.63806', '628.333', '1084.29', 1404904442.549, 18.000125885009766]
---------
['teacher', '5.63806', '627.125', '1080.38', 1404904442.565, 16.000032424926758]
---------
['teacher', '5.63806', '627.125', '1076.22', 1404904442.581, 16.000032424926758]
---------
['teacher', '5.63806', '625.333', '1068.44', 1404904442.598, 16.99995994567871]
---------
['teacher', '5.63806', '625.333', '1060.67', 1404904442.618, 19.999980926513672]
---------
['teacher', '5.63806', '625.333', '1052.89', 1404904442.632, 13.99993896484375]
---------
['teac

---------
['A', '1.0472', '358', '1407.44', 1404904494.527, 16.000032424926758]
---------
['A', '1.0472', '356.778', '1418', 1404904494.544, 16.99995994567871]
---------
['A', '1.0472', '355.111', '1428.56', 1404904494.561, 17.000198364257812]
---------
['A', '1.0472', '353.444', '1438.89', 1404904494.577, 15.999794006347656]
---------
['A', '1.0472', '351.444', '1448.78', 1404904494.594, 17.000198364257812]
---------
['A', '1.0472', '349.222', '1459', 1404904494.611, 16.99995994567871]
---------
['A', '1.0472', '346.778', '1469.22', 1404904494.627, 16.000032424926758]
---------
['A', '0.881873', '344.222', '1478.67', 1404904494.644, 16.99995994567871]
---------
['A', '0.881873', '341.444', '1487.89', 1404904494.661, 16.99995994567871]
---------
['A', '0.881873', '335.222', '1504.89', 1404904494.694, 6.999969482421875]
---------
['A', '0.881873', '332.444', '1512.56', 1404904494.711, 16.99995994567871]
---------
['A', '0.881873', '329.111', '1519.67', 1404904494.729, 18.000125885009766

---------
['A', '1.61339', '147.444', '1082.89', 1404904582.641, 2.000093460083008]
---------
['A', '2.66907', '70', '1051', 1404904582.807, 6.999969482421875]
---------
['A', '2.52301', '71.2', '1055.2', 1404904582.857, 18.000125885009766]
---------
['A', '2.51392', '199', '1067', 1404904583.073, 16.99995994567871]
---------
['A', '2.51392', '199', '1066', 1404904583.089, 16.000032424926758]
---------
['A', '2.27124', '203', '1116', 1404904583.273, 3.999948501586914]
---------
['A', '2.27124', '202', '1117.33', 1404904583.289, 16.000032424926758]
---------
['A', '2.27124', '201', '1117.33', 1404904583.308, 19.00005340576172]
---------
['A', '2.44336', '154', '1143', 1404904583.489, 15.000104904174805]
---------
['A', '1.17349', '191', '851', 1404904583.79, 16.99995994567871]
---------
['A', '2.17224', '199', '1106', 1404904584.507, 16.99995994567871]
---------
['A', '2.48627', '149', '1130', 1404904584.724, 16.99995994567871]
---------
['A', '0.366905', '63', '956', 1404904584.959, 7.

---------
['A', '1.27035', '239', '675', 1404904701.429, 1.9998550415039062]
---------
['A', '1.27035', '239', '675', 1404904701.432, 3.000020980834961]
---------
['A', '1.27035', '239', '675', 1404904701.451, 19.00005340576172]
---------
['A', '1.27035', '239', '675', 1404904701.453, 2.000093460083008]
---------
['A', '1.27035', '239', '675', 1404904701.461, 7.999897003173828]
---------
['A', '1.27035', '239', '675', 1404904701.463, 2.000093460083008]
---------
['A', '0.868011', '148', '867', 1404904702.817, 18.999814987182617]
---------
['A', '1.4964', '242', '678', 1404904703.834, 20.000219345092773]
---------
['A', '1.4964', '241', '678', 1404904703.884, 21.000146865844727]
---------
['A', '1.34341', '193', '794', 1404904704.882, 16.99995994567871]
---------
['A', '2.69214', '69', '1026', 1404904705.081, 16.000032424926758]
---------
['A', '2.01379', '733', '1089.5', 1404904706.569, 19.999980926513672]
---------
['A', '1.86613', '733', '1089.5', 1404904706.583, 13.99993896484375]
-

['A', '1.62859', '241', '667.111', 1404904851.032, 2.000093460083008]
---------
['A', '1.62859', '241', '667.111', 1404904851.033, 0.9999275207519531]
---------
['A', '1.62859', '241', '667.111', 1404904851.036, 3.000020980834961]
---------
['A', '1.62859', '241', '667.111', 1404904851.038, 2.000093460083008]
---------
['A', '1.62859', '241', '667.111', 1404904851.041, 2.9997825622558594]
---------
['A', '1.62859', '241', '667.111', 1404904851.043, 2.000093460083008]
---------
['A', '1.62859', '241', '667.111', 1404904851.046, 3.000020980834961]
---------
['A', '1.62859', '242.111', '667.111', 1404904851.048, 2.000093460083008]
---------
['A', '1.62859', '242.111', '667.111', 1404904851.051, 3.000020980834961]
---------
['A', '1.62859', '242.111', '667.111', 1404904851.053, 1.9998550415039062]
---------
['A', '1.62859', '242.111', '667.111', 1404904851.057, 3.999948501586914]
---------
['A', '1.62859', '242.111', '667.111', 1404904851.06, 3.000020980834961]
---------
['A', '1.62859', '

['A', '1.40135', '390.111', '550', 1404904921.398, 2.000093460083008]
---------
['A', '1.40135', '390.111', '550', 1404904921.4, 2.000093460083008]
---------
['A', '1.40135', '388.444', '544.778', 1404904921.406, 5.99980354309082]
---------
['A', '1.40135', '388.444', '544.778', 1404904921.409, 3.000020980834961]
---------
['A', '1.40135', '388.444', '544.778', 1404904921.411, 2.000093460083008]
---------
['A', '1.40135', '388.444', '544.778', 1404904921.413, 2.000093460083008]
---------
['A', '1.40135', '386.778', '539.333', 1404904921.422, 8.999824523925781]
---------
['A', '1.40135', '386.778', '539.333', 1404904921.434, 12.000083923339844]
---------
['A', '1.40135', '386.778', '539.333', 1404904921.436, 2.000093460083008]
---------
['A', '1.40135', '386.778', '539.333', 1404904921.438, 1.9998550415039062]
---------
['A', '1.54842', '385.222', '534.555', 1404904921.442, 3.999948501586914]
---------
['A', '1.54842', '385.222', '534.555', 1404904921.444, 2.000093460083008]
---------
[

In [2]:
lastPoints

[['teacher', '5.68721', '963.222', '1855.11', 1404904960.757, 4550],
 ['A', '3.2238', '66', '1064.25', 1404904950.366, 4489],
 ['B', '3.84158', '957.5', '345.25', 1404904808.743, 2910],
 ['C', '0.871491', '306.333', '317.222', 1404904806.271, 2885]]

In [3]:
len(touchPairs)

12806

In [6]:
deltaTimeThreshold

114275.00009536743